# Chapter 45: Distributed Tracing

While metrics reveal what is happening (high latency, error rates) and logs reveal why (error messages, stack traces), distributed tracing reveals where latency originates in complex, multi-service requests. When a user clicks "Checkout" on an e-commerce site, the request may traverse the API Gateway → Authentication Service → Cart Service → Inventory Service → Payment Service → Notification Service → Email Provider. A 500ms delay observed in metrics could originate in any of these hops, or in the network between them. Distributed tracing captures the complete request journey, breaking it into spans—timed operations representing individual units of work—connected by a trace ID that binds them into a single logical transaction.

This chapter explores OpenTelemetry, the emerging industry standard for telemetry instrumentation, covering trace propagation across service boundaries, sampling strategies that balance observability overhead with data volume, and visualization tools that transform span data into actionable latency analysis.

## 45.1 OpenTelemetry Fundamentals

OpenTelemetry (OTel) is a Cloud Native Computing Foundation (CNCF) project that provides standardized APIs, libraries, agents, and collector services for capturing distributed traces, metrics, and logs. It supersedes previous standards like OpenTracing and OpenCensus, offering a unified, vendor-neutral instrumentation layer.

### Architecture Components

**API and SDK**
The OpenTelemetry API defines interfaces for creating spans, adding attributes, and propagating context. The SDK implements these interfaces, handling batching, sampling, and export configuration. Applications depend only on the API, while operators configure the SDK via environment variables or code.

**Instrumentation Libraries**
Auto-instrumentation agents (Java agent, Python auto-instrumentation) attach to frameworks (Spring Boot, Django, Express) without code changes, automatically creating spans for HTTP requests, database queries, and message queue operations.

**Collector**
The OpenTelemetry Collector receives telemetry data (traces, metrics, logs), processes it (batching, filtering, enriching), and exports it to backends (Jaeger, Zipkin, Prometheus, commercial vendors). It operates as an agent (sidecar or DaemonSet) or gateway (cluster-wide service).

**Protocol (OTLP)**
The OpenTelemetry Protocol defines a standardized wire format for telemetry data, supporting gRPC and HTTP/protobuf or HTTP/JSON transports.

### Java Auto-Instrumentation

```java
// No code changes required for basic tracing
// Add JVM argument:
// -javaagent:opentelemetry-javaagent.jar \
// -Dotel.service.name=payment-service \
// -Dotel.traces.exporter=otlp \
// -Dotel.exporter.otlp.endpoint=http://otel-collector:4317

// Custom span for business logic
import io.opentelemetry.api.GlobalOpenTelemetry;
import io.opentelemetry.api.trace.Span;
import io.opentelemetry.api.trace.Tracer;
import io.opentelemetry.context.Scope;

@Service
public class PaymentProcessor {
    private final Tracer tracer;
    
    public PaymentProcessor() {
        // Get tracer from global singleton
        this.tracer = GlobalOpenTelemetry.getTracer("payment-service", "1.0.0");
    }
    
    public PaymentResult processPayment(PaymentRequest request) {
        // Create a span representing this operation
        Span span = tracer.spanBuilder("process-payment")
            .setAttribute("payment.id", request.getPaymentId())
            .setAttribute("payment.amount", request.getAmount().doubleValue())
            .setAttribute("payment.currency", request.getCurrency())
            .setAttribute("user.id", request.getUserId())
            .startSpan();
        
        // Make this span the current span in this thread's context
        try (Scope scope = span.makeCurrent()) {
            
            // Add events (timestamps with descriptions)
            span.addEvent("Validating payment request");
            validateRequest(request);
            
            span.addEvent("Reserving inventory");
            inventoryService.reserve(request.getItems());
            
            span.addEvent("Processing charge");
            ChargeResult charge = paymentGateway.charge(request);
            
            span.setAttribute("payment.status", charge.getStatus());
            span.setAttribute("payment.gateway_transaction_id", charge.getTransactionId());
            
            if (charge.isSuccessful()) {
                span.setStatus(StatusCode.OK);
            } else {
                span.setStatus(StatusCode.ERROR, "Payment declined: " + charge.getDeclineReason());
            }
            
            return new PaymentResult(charge);
            
        } catch (Exception e) {
            // Record exception with stack trace
            span.recordException(e);
            span.setStatus(StatusCode.ERROR, "Payment processing failed");
            throw e;
        } finally {
            // End span (records duration)
            span.end();
        }
    }
}
```

**Explanation:**
- **GlobalOpenTelemetry**: Access point to the configured SDK. In auto-instrumentation, this is initialized by the Java agent using environment variables.
- **Tracer**: Creates spans. The instrumentation scope name ("payment-service") identifies which library generated the spans.
- **SpanBuilder**: Configures the span before creation. Attributes added here are key-value pairs (string, number, boolean) attached to the span.
- **makeCurrent()**: Places the span in thread-local storage so child operations (HTTP calls, database queries) automatically create child spans linked to this parent.
- **Scope**: Auto-closable resource that removes the span from context when done (try-with-resources ensures cleanup even on exceptions).
- **Events**: Timestamped annotations within the span timeline, useful for marking sub-operations.
- **Status**: Explicit success/error indication. Errors include descriptions and optionally stack traces.

### Manual Context Propagation

When crossing asynchronous boundaries or non-standard transports:

```java
import io.opentelemetry.context.Context;

@Service
public class AsyncPaymentProcessor {
    
    public CompletableFuture<PaymentResult> processAsync(PaymentRequest request) {
        // Capture current context (contains current span if any)
        Context parentContext = Context.current();
        
        return CompletableFuture.supplyAsync(() -> {
            // Wrap runnable with parent context
            try (Scope scope = parentContext.makeCurrent()) {
                // Now inside the async thread, context is restored
                // Any spans created here will be children of the parent
                return processPayment(request);
            }
        });
    }
}
```

**Explanation:**
When using `CompletableFuture` or thread pools, the `Context` doesn't automatically propagate across threads (unlike `ThreadLocal`). The parent context must be captured before the async operation and explicitly made current in the new thread. This ensures trace continuity across asynchronous boundaries.

### Python Instrumentation

```python
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import SERVICE_NAME, Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor

# Configure SDK
resource = Resource(attributes={SERVICE_NAME: "order-service"})
provider = TracerProvider(resource=resource)
processor = BatchSpanProcessor(OTLPSpanExporter(endpoint="otel-collector:4317"))
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)

tracer = trace.get_tracer("order-service", "1.0.0")

class OrderService:
    def create_order(self, request):
        # Create span
        with tracer.start_as_current_span("create-order") as span:
            # Set attributes
            span.set_attribute("order.id", request.order_id)
            span.set_attribute("user.id", request.user_id)
            span.set_attribute("item.count", len(request.items))
            
            # Add event
            span.add_event("Validating inventory")
            
            try:
                # This HTTP call will be auto-instrumented if using requests instrumentation
                response = self.inventory_client.check_availability(request.items)
                
                span.set_attribute("inventory.available", response.available)
                
                if not response.available:
                    span.set_status(trace.Status(trace.StatusCode.ERROR, "Items out of stock"))
                    raise OutOfStockError()
                
                span.add_event("Creating order record")
                order = self.db.save_order(request)
                
                span.set_attribute("order.total", float(order.total))
                span.set_status(trace.Status(trace.StatusCode.OK))
                
                return order
                
            except Exception as e:
                span.record_exception(e)
                span.set_status(trace.Status(trace.StatusCode.ERROR, str(e)))
                raise
```

**Explanation:**
The Python SDK uses context managers (`with` statement) for span lifecycle management. `start_as_current_span` automatically makes the span current in the context manager scope. Attributes use snake_case by convention in Python.

## 45.2 Trace Context Propagation

For distributed traces to span multiple services, trace context must propagate across network boundaries via standardized headers.

### W3C Trace Context

The W3C standard defines two headers:
- `traceparent`: Describes the incoming request's position in the trace (trace ID, span ID, flags)
- `tracestate**: Vendor-specific extensions

**Traceparent Format:**
```
version-trace_id-span_id-flags
00-0af7651916cd43dd8448eb211c80319c-b7ad6b7169203331-01
```

**Breakdown:**
- `00`: Version
- `0af7651916cd43dd8448eb211c80319c`: 16-byte hex trace ID (unique request identifier)
- `b7ad6b7169203331`: 16-byte hex span ID (current operation identifier)
- `01`: Flags (bit 0 = sampled)

### HTTP Client Propagation

**Java (Spring WebClient):**
```java
import io.opentelemetry.api.GlobalOpenTelemetry;
import io.opentelemetry.context.propagation.ContextPropagators;
import io.opentelemetry.context.propagation.TextMapPropagator;
import org.springframework.web.reactive.function.client.ClientRequest;
import org.springframework.web.reactive.function.client.WebClient;

@Service
public class PaymentClient {
    private final WebClient webClient;
    private final TextMapPropagator propagator;
    
    public PaymentClient(WebClient.Builder builder) {
        this.webClient = builder.baseUrl("http://payment-service").build();
        // Get W3C propagator from global configuration
        this.propagator = GlobalOpenTelemetry.getPropagators().getTextMapPropagator();
    }
    
    public Mono<PaymentResult> charge(PaymentRequest request) {
        return webClient.post()
            .uri("/charge")
            .bodyValue(request)
            .httpRequest(httpRequest -> {
                // Inject current context into HTTP headers
                propagator.inject(
                    Context.current(),
                    httpRequest.getHeaders(),
                    (carrier, key, value) -> carrier.add(key, value)
                );
            })
            .retrieve()
            .bodyToMono(PaymentResult.class);
    }
}
```

**Explanation:**
The `inject` method takes the current context (containing the active span), the carrier (HTTP headers), and a setter function that knows how to add headers to the carrier. This adds `traceparent` and `tracestate` headers to the outgoing HTTP request. The downstream service extracts these headers to continue the trace.

### HTTP Server Extraction

```java
import io.opentelemetry.api.trace.Span;
import io.opentelemetry.api.trace.SpanKind;
import io.opentelemetry.context.Context;
import io.opentelemetry.context.propagation.TextMapGetter;
import org.springframework.stereotype.Component;

import javax.servlet.Filter;
import javax.servlet.FilterChain;
import javax.servlet.ServletException;
import javax.servlet.ServletRequest;
import javax.servlet.ServletResponse;
import javax.servlet.http.HttpServletRequest;
import java.io.IOException;

@Component
public class TracingFilter implements Filter {
    private final TextMapPropagator propagator;
    
    public TracingFilter() {
        this.propagator = GlobalOpenTelemetry.getPropagators().getTextMapPropagator();
    }
    
    @Override
    public void doFilter(ServletRequest request, ServletResponse response, 
                        FilterChain chain) throws IOException, ServletException {
        
        HttpServletRequest httpRequest = (HttpServletRequest) request;
        
        // Extract context from incoming headers
        Context extractedContext = propagator.extract(
            Context.current(),
            httpRequest,
            new TextMapGetter<HttpServletRequest>() {
                @Override
                public Iterable<String> keys(HttpServletRequest carrier) {
                    return Collections.list(carrier.getHeaderNames());
                }
                
                @Override
                public String get(HttpServletRequest carrier, String key) {
                    return carrier.getHeader(key);
                }
            }
        );
        
        // Create a new span as child of extracted context
        Span span = GlobalOpenTelemetry.getTracer("order-service")
            .spanBuilder(httpRequest.getRequestURI())
            .setParent(extractedContext)
            .setSpanKind(SpanKind.SERVER)
            .setAttribute("http.method", httpRequest.getMethod())
            .setAttribute("http.url", httpRequest.getRequestURL().toString())
            .startSpan();
        
        try (Scope scope = span.makeCurrent()) {
            chain.doFilter(request, response);
        } finally {
            span.end();
        }
    }
}
```

**Explanation:**
The server extracts the `traceparent` header using the getter interface. `extractedContext` contains the parent span ID from the upstream service. The new span created is a child of that parent, linking them in the trace hierarchy. `SpanKind.SERVER` indicates this is an incoming request handling span.

### Message Queue Propagation (Kafka)

```java
import org.apache.kafka.clients.producer.ProducerRecord;
import io.opentelemetry.api.GlobalOpenTelemetry;

@Service
public class EventPublisher {
    private final KafkaTemplate<String, String> kafkaTemplate;
    private final TextMapPropagator propagator;
    
    public void publishOrderCreated(Order order) {
        ProducerRecord<String, String> record = new ProducerRecord<>(
            "orders.created", 
            order.getId(), 
            toJson(order)
        );
        
        // Inject context into Kafka headers
        propagator.inject(
            Context.current(),
            record.headers(),
            (headers, key, value) -> headers.add(key, value.getBytes(StandardCharsets.UTF_8))
        );
        
        kafkaTemplate.send(record);
    }
}

// Consumer side
@KafkaListener(topics = "orders.created")
public void handleOrderCreated(ConsumerRecord<String, String> record) {
    // Extract from Kafka headers
    Context extractedContext = propagator.extract(
        Context.current(),
        record.headers(),
        new TextMapGetter<Headers>() {
            @Override
            public String get(Headers headers, String key) {
                Header header = headers.lastHeader(key);
                return header != null ? new String(header.value(), StandardCharsets.UTF_8) : null;
            }
            
            @Override
            public Iterable<String> keys(Headers headers) {
                List<String> keys = new ArrayList<>();
                headers.forEach(h -> keys.add(h.key()));
                return keys;
            }
        }
    );
    
    // Create span continuing the trace
    Span span = tracer.spanBuilder("process-order-event")
        .setParent(extractedContext)
        .setAttribute("kafka.topic", record.topic())
        .setAttribute("kafka.partition", record.partition())
        .setAttribute("kafka.offset", record.offset())
        .startSpan();
    
    try (Scope scope = span.makeCurrent()) {
        processOrder(record.value());
    } finally {
        span.end();
    }
}
```

**Explanation:**
Kafka headers (supported in 0.11+) carry trace context. The producer injects context into headers; the consumer extracts it. This maintains trace continuity even across asynchronous message queues, ensuring that an order created in the HTTP request trace continues into the background processing trace.

## 45.3 Sampling Strategies

Tracing every request in high-throughput systems (10,000+ RPS) generates prohibitive data volumes and costs. Sampling reduces data volume while maintaining statistical representativeness.

### Head-Based Sampling

The sampling decision is made at the start of the trace and propagated to all child spans.

**Configuration:**
```yaml
# otel-collector-config.yaml
processors:
  probabilistic_sampler:
    sampling_percentage: 10.0  # Sample 10% of traces
    hash_seed: 22
    
  tail_sampling:  # Requires waiting for spans, more complex
    decision_wait: 10s
    num_traces: 100
    expected_new_traces_per_sec: 10
    policies:
      - name: errors
        type: status_code
        status_code: {status_codes: [ERROR]}
      - name: slow_requests
        type: latency
        latency: {threshold_ms: 1000}
```

**Environment Variables:**
```bash
# Java application
export OTEL_TRACES_SAMPLER=traceidratio
export OTEL_TRACES_SAMPLER_ARG=0.1  # 10% sampling
```

**Explanation:**
Head-based sampling is efficient (decision made once, no buffering required) but may miss interesting traces. If you sample based on trace ID hash, and the error happens only in specific trace IDs, you might miss all errors. However, it's stateless and suitable for most high-volume scenarios.

### Tail-Based Sampling

The sampling decision is made after the trace completes, based on full trace characteristics (errors, latency).

```yaml
# OpenTelemetry Collector configuration
processors:
  tail_sampling:
    decision_wait: 10s  # Wait for spans to arrive
    num_traces: 100000  # Buffer size
    expected_new_traces_per_sec: 1000
    
    policies:
      # Sample all errors
      - name: errors
        type: status_code
        status_code: {status_codes: [ERROR]}
        
      # Sample slow traces (>1s)
      - name: slow
        type: latency
        latency: {threshold_ms: 1000}
        
      # Sample specific operations
      - name: important_ops
        type: string_attribute
        string_attribute:
          key: http.route
          values: ["/api/v1/payments", "/api/v1/orders"]
          enabled_regex_matching: false
          
      # Probabilistic for remainder
      - name: probabilistic
        type: probabilistic
        probabilistic: {sampling_percentage: 5}
```

**Explanation:**
Tail-based sampling buffers spans for `decision_wait` seconds, then evaluates policies. If any policy matches (error occurred, latency > 1s, specific route), the entire trace is kept; otherwise, it's dropped. This captures 100% of errors while sampling healthy traffic at 5%. The trade-off is memory usage (buffering spans) and slight delay in export.

### Parent-Based Sampling

Respect the parent's sampling decision:

```java
// If parent span was sampled, sample this span
// If parent was not sampled, don't sample this span
Tracer tracer = GlobalOpenTelemetry.getTracer("service");
Span span = tracer.spanBuilder("child-operation")
    .setParent(parentContext)
    .setAttribute("sampling.priority", 10)  // Hint for some samplers
    .startSpan();
```

**Explanation:**
Ensures trace consistency—all spans in a trace are either sampled or not. Without this, you might have a root span sampled but child spans dropped, resulting in broken traces in the backend.

## 45.4 Baggage

Baggage carries user-defined key-value pairs across service boundaries alongside trace context, useful for propagating tenant IDs, feature flags, or user context without adding to every function parameter.

### Setting and Propagating Baggage

```java
import io.opentelemetry.api.baggage.Baggage;
import io.opentelemetry.api.baggage.BaggageEntry;

// In the API Gateway (entry point)
public void handleRequest(HttpServletRequest request) {
    // Extract tenant from JWT or header
    String tenantId = extractTenant(request);
    String featureFlag = request.getHeader("X-Feature-Flag");
    
    // Create baggage
    Baggage baggage = Baggage.builder()
        .put("tenant.id", tenantId)
        .put("tenant.tier", getTenantTier(tenantId))
        .put("feature.new_checkout", featureFlag)
        .build();
    
    // Make baggage current (stored in context)
    try (Scope baggageScope = baggage.makeCurrent()) {
        // Process request - baggage automatically propagates
        processRequest(request);
    }
}

// In downstream service (automatically receives baggage)
public void chargePayment(PaymentRequest request) {
    // Access baggage without explicit parameter passing
    Baggage baggage = Baggage.current();
    String tenantId = baggage.getEntryValue("tenant.id");
    String tier = baggage.getEntryValue("tenant.tier");
    
    // Add to span for filtering/analysis
    Span.current().setAttribute("tenant.id", tenantId);
    Span.current().setAttribute("tenant.tier", tier);
    
    // Use for business logic
    if ("enterprise".equals(tier)) {
        // Priority processing
    }
}
```

**Explanation:**
Baggage is separate from span attributes—it travels with the context across process boundaries via the same propagation mechanism as trace context (W3C baggage header). Unlike span attributes which describe the operation, baggage describes the request context (who is making the request, what features are enabled).

**Propagation Format:**
```
Baggage: tenant.id=acme-corp,tenant.tier=enterprise,feature.new_checkout=enabled
```

### Python Baggage Example

```python
from opentelemetry import baggage
from opentelemetry.propagate import extract, inject, set_global_textmap
from opentelemetry.propagators.b3 import B3Format

# Set baggage
ctx = baggage.set_baggage("tenant.id", "12345")
ctx = baggage.set_baggage("user.type", "premium", context=ctx)

# Make current
token = context.attach(ctx)

try:
    # HTTP call automatically includes baggage headers
    response = requests.get("http://downstream-service/api", 
                          headers=inject_current_context())
finally:
    context.detach(token)

# In downstream service
current_baggage = baggage.get_baggage("tenant.id")
span.set_attribute("tenant.id", current_baggage)
```

## 45.5 Correlation with Logs and Metrics

### Trace ID in Logs

Ensure every log entry includes the trace ID for correlation:

```java
// Logback configuration (logback-spring.xml)
<appender name="JSON_CONSOLE" class="ch.qos.logback.core.ConsoleAppender">
    <encoder class="net.logstash.logback.encoder.LogstashEncoder">
        <includeMdc>true</includeMdc>
        <provider class="net.logstash.logback.composite.loggingevent.MdcJsonProvider">
            <includeMdcKeyName>trace_id</includeMdcKeyName>
            <includeMdcKeyName>span_id</includeMdcKeyName>
        </provider>
    </encoder>
</appender>

// Code to ensure trace context is in MDC
import io.opentelemetry.api.trace.Span;
import io.opentelemetry.api.trace.SpanContext;
import org.slf4j.MDC;

public class TraceContextFilter implements Filter {
    @Override
    public void doFilter(ServletRequest request, ServletResponse response, 
                        FilterChain chain) throws IOException, ServletException {
        
        SpanContext spanContext = Span.current().getSpanContext();
        
        if (spanContext.isValid()) {
            MDC.put("trace_id", spanContext.getTraceId());
            MDC.put("span_id", spanContext.getSpanId());
            MDC.put("trace_flags", spanContext.getTraceFlags().toString());
        }
        
        try {
            chain.doFilter(request, response);
        } finally {
            MDC.remove("trace_id");
            MDC.remove("span_id");
            MDC.remove("trace_flags");
        }
    }
}
```

**Explanation:**
The filter extracts the trace ID from the current OpenTelemetry span and places it in MDC (Mapped Diagnostic Context). Logback's JSON encoder includes MDC fields in the log output. Now logs and traces are linked: find a slow trace ID in Jaeger, search for that ID in Kibana/Loki to see application logs for that exact request.

### Exemplars (Trace-Metric Link)

Exemplars link metrics to example traces:

```java
// Micrometer with OpenTelemetry integration
import io.micrometer.core.instrument.Timer;
import io.micrometer.core.instrument.MeterRegistry;

@Component
public class PaymentMetrics {
    private final Timer paymentTimer;
    
    public PaymentMetrics(MeterRegistry registry) {
        this.paymentTimer = Timer.builder("payment.duration")
            .description("Payment processing time")
            .register(registry);
    }
    
    public void recordPayment(PaymentRequest request, long durationMs) {
        // Micrometer automatically attaches current span context as exemplar
        paymentTimer.record(durationMs, TimeUnit.MILLISECONDS);
    }
}
```

**Prometheus Configuration:**
```yaml
# Enable exemplars in Prometheus
storage:
  tsdb:
    exemplars:
      max_size: 10000000
```

**Explanation:**
When Prometheus scrapes the `/actuator/prometheus` endpoint, Micrometer includes exemplars—references to specific trace IDs that contributed to the metric value. In Grafana, when viewing a histogram of latency, you can click on a data point and see "Example Trace" linking to Jaeger. This bridges aggregated metrics and individual traces.

## 45.6 Visualization with Jaeger

Jaeger is an open-source distributed tracing system inspired by Google's Dapper.

### Jaeger Architecture

```
Application → OpenTelemetry SDK → OTLP → Jaeger Collector → Storage (Elasticsearch/Cassandra/Badger) → Jaeger Query → Jaeger UI
```

### Deployment

```yaml
# jaeger-deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: jaeger
spec:
  replicas: 1
  selector:
    matchLabels:
      app: jaeger
  template:
    metadata:
      labels:
        app: jaeger
    spec:
      containers:
        - name: jaeger
          image: jaegertracing/all-in-one:1.50
          env:
            - name: COLLECTOR_OTLP_ENABLED
              value: "true"
          ports:
            - containerPort: 16686  # UI
            - containerPort: 4317   # OTLP gRPC
            - containerPort: 4318   # OTLP HTTP
---
apiVersion: v1
kind: Service
metadata:
  name: jaeger-query
spec:
  selector:
    app: jaeger
  ports:
    - port: 16686
      targetPort: 16686
```

### Using the Jaeger UI

**Trace View:**
- **Timeline**: Shows spans horizontally with duration bars. Long bars indicate slow operations.
- **Service Dependencies**: Graph showing which services call which others.
- **Trace Comparison**: Compare two traces side-by-side to find performance regressions.

**Search Query:**
```
service=payment-service 
operation=process-payment 
tags={"http.status_code":500} 
duration>1s 
start=2024-01-15T10:00:00 
end=2024-01-15T11:00:00
```

**Explanation:**
Search for traces where the payment-service had errors (status 500) and duration > 1 second in a specific time range. The results show the full trace, revealing whether the slowness was in the database (span name "postgres-query" duration 900ms) or external API ("stripe-api" duration 100ms).

## 45.7 Performance Considerations

### Overhead Management

Tracing adds overhead: CPU for serialization, memory for buffering, network for export.

**Optimization Strategies:**

```yaml
# Batch export configuration
exporters:
  otlp:
    endpoint: otel-collector:4317
    timeout: 10s
    sending_queue:
      enabled: true
      num_consumers: 10
      queue_size: 1000
    retry_on_failure:
      enabled: true
      initial_interval: 5s
      max_interval: 30s
      max_elapsed_time: 300s
```

**Explanation:**
- **Batching**: Export spans in batches (e.g., every 1 second or 512 spans) rather than one-by-one to reduce network calls.
- **Queue**: Asynchronous export prevents application blocking if collector is slow.
- **Retry**: Handles transient network failures without losing data.

### Sampling in High-Throughput Services

For services handling 50,000+ RPS:

```java
// Adaptive sampling based on load
Tracer tracer = TracerProvider.builder()
    .setSampler(Sampler.traceIdRatioBased(0.01))  // 1% sampling at high load
    .build()
    .getTracer("high-volume-service");
```

Or use the OpenTelemetry Collector to sample centrally:

```yaml
processors:
  probabilistic_sampler:
    sampling_percentage: 1.0  # Only sample 1% at collector level
```

### Context Propagation Cost

Avoid propagating huge baggage:

```java
// Bad - large payload in baggage
Baggage.builder()
    .put("user.data", largeJsonString)  // Don't do this
    .build();

// Good - minimal context
Baggage.builder()
    .put("user.id", userId)  // Just the ID
    .build();
```

**Explanation:**
Baggage travels in HTTP headers on every outbound request. Large baggage increases header size, potentially exceeding limits (8KB common in load balancers) and adding latency.

---

## Chapter Summary and Preview

This chapter established distributed tracing as the third pillar of observability, complementing logs and metrics by revealing request flow and latency decomposition across microservices. We examined OpenTelemetry as the unified instrumentation standard, covering the API/SDK separation that allows vendor-neutral instrumentation, auto-instrumentation agents that capture framework-level operations without code changes, and the OpenTelemetry Protocol (OTLP) for efficient data transport.

Trace context propagation via W3C Trace Context ensures requests maintain their identity across HTTP calls and message queues, enabling complete request chains from edge to database. We explored head-based sampling for efficient high-volume tracing decisions, tail-based sampling for intelligent retention of error or slow traces, and baggage for propagating business context (tenant IDs, feature flags) without polluting function signatures. The integration of traces with logs (via trace IDs in MDC) and metrics (via exemplars) creates a unified observability experience where quantitative anomalies in metrics lead to representative traces and detailed logs.

Visualization tools like Jaeger transform span data into dependency graphs and timelines that make latency bottlenecks immediately visible. Performance considerations including batch export, queue-based asynchronous transmission, and appropriate sampling rates ensure tracing overhead remains below 5% even in high-throughput systems.

**Key Takeaways:**
- Adopt OpenTelemetry for all new instrumentation, using auto-instrumentation agents for existing applications to capture traces without code modification, and manual span creation only for critical business operations requiring custom attributes.
- Always propagate W3C Trace Context headers (traceparent) in HTTP clients and message producers; ensure downstream services extract these headers to maintain trace continuity across service boundaries.
- Use tail-based sampling in the OpenTelemetry Collector to retain 100% of error traces and slow requests (>1s) while sampling healthy traffic at 1-5%, balancing observability depth with storage costs.
- Include trace_id and span_id in structured logs using MDC or equivalent context mechanisms, enabling correlation between traces and logs for root cause analysis.
- Use baggage sparingly for lightweight context propagation (tenant IDs, user types), never for large payloads, and always add baggage values to span attributes at service boundaries for searchability.
- Configure batch span processors with queue sizes appropriate for your throughput (typically 2048-4096 spans) and export intervals (1-5 seconds) to minimize network overhead while preventing memory pressure.

**Next Chapter Preview:**
Chapter 46: Debugging CI/CD Pipelines addresses the operational challenges of maintaining complex continuous delivery systems. We will explore strategies for diagnosing pipeline failures, container debugging techniques (exec, debug containers, ephemeral containers), Kubernetes pod troubleshooting (crash loops, image pull errors, resource constraints), network debugging across clusters, and permission issues in CI environments. The chapter covers log aggregation from CI runners, build reproducibility strategies, and tools for tracing pipeline execution across multiple stages and repositories, ensuring that when deployments fail, teams can rapidly identify whether the issue lies in code, configuration, infrastructure, or the pipeline itself.